In [1]:
import os
import torch
import torch.nn as nn
import math
import time
import tarfile
import html
import random
import numpy as np
from collections import Counter
from torch.utils.data import DataLoader, Dataset
from google.colab import drive

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

PROJECT_PATH = '/content/drive/MyDrive/NLP_Phase5_Final'
CHECKPOINT_PATH = os.path.join(PROJECT_PATH, 'checkpoints')
DATA_SAVE_PATH = os.path.join(PROJECT_PATH, 'data')

if not os.path.exists(CHECKPOINT_PATH): os.makedirs(CHECKPOINT_PATH)
if not os.path.exists(DATA_SAVE_PATH): os.makedirs(DATA_SAVE_PATH)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on: {DEVICE}")

def download_and_clean():
    url = "https://github.com/stefan-it/nmt-en-vi/raw/master/data/train-en-vi.tgz"
    tgz_path = os.path.join(DATA_SAVE_PATH, "train-en-vi.tgz")

    if not os.path.exists(os.path.join(DATA_SAVE_PATH, "train.vi")):
        print("Downloading data...")
        os.system(f"wget -q {url} -O {tgz_path}")
        with tarfile.open(tgz_path, "r:gz") as tar:
            tar.extractall(path=DATA_SAVE_PATH)

    def clean_data(path):
        with open(path, 'r', encoding='utf-8') as f:
            return [html.unescape(line).replace('\xa0', ' ').strip() for line in f]

    src_lines = clean_data(os.path.join(DATA_SAVE_PATH, 'train.vi'))
    trg_lines = clean_data(os.path.join(DATA_SAVE_PATH, 'train.en'))
    return src_lines, trg_lines

src_raw, trg_raw = download_and_clean()
print(f"Loaded {len(src_raw)} sentences.")

def tokenize(text):
    return text.lower().split()

class Vocabulary:
    def __init__(self, freq_threshold=2):
        self.itos = {0: "<pad>", 1: "<sos>", 2: "<eos>", 3: "<unk>"}
        self.stoi = {"<pad>": 0, "<sos>": 1, "<eos>": 2, "<unk>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self): return len(self.itos)

    def build_vocabulary(self, sentence_list):
        frequencies = Counter()
        for sentence in sentence_list:
            frequencies.update(tokenize(sentence))

        idx = 4
        for word, freq in frequencies.items():
            if freq >= self.freq_threshold:
                self.stoi[word] = idx
                self.itos[idx] = word
                idx += 1

    def numericalize(self, text):
        return [self.stoi.get(token, self.stoi["<unk>"]) for token in tokenize(text)]

print("Building Vocab...")
src_vocab = Vocabulary(freq_threshold=2)
trg_vocab = Vocabulary(freq_threshold=2)
src_vocab.build_vocabulary(src_raw)
trg_vocab.build_vocabulary(trg_raw)
print(f"Vocab Size: Vi={len(src_vocab)}, En={len(trg_vocab)}")

# 4. Dataset & DataLoader
class TranslationDataset(Dataset):
    def __init__(self, src_lines, trg_lines, src_vocab, trg_vocab, max_len=100):
        self.src_lines = src_lines
        self.trg_lines = trg_lines
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab
        self.max_len = max_len

    def __len__(self): return len(self.src_lines)

    def __getitem__(self, idx):
        src_idx = [1] + self.src_vocab.numericalize(self.src_lines[idx]) + [2]
        trg_idx = [1] + self.trg_vocab.numericalize(self.trg_lines[idx]) + [2]
        # Cắt nếu quá dài
        return torch.tensor(src_idx[:self.max_len]), torch.tensor(trg_idx[:self.max_len])

def collate_fn(batch):
    src_batch, trg_batch = [], []
    for src, trg in batch:
        src_batch.append(src)
        trg_batch.append(trg)
    src_pad = torch.nn.utils.rnn.pad_sequence(src_batch, padding_value=0, batch_first=True)
    trg_pad = torch.nn.utils.rnn.pad_sequence(trg_batch, padding_value=0, batch_first=True)
    return src_pad, trg_pad

train_loader = DataLoader(TranslationDataset(src_raw, trg_raw, src_vocab, trg_vocab),
                          batch_size=32, shuffle=True, collate_fn=collate_fn)

Mounted at /content/drive
Running on: cuda


/tmp/ipython-input-163462796.py:35: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=DATA_SAVE_PATH)


Loaded 133317 sentences.
Building Vocab...
Vocab Size: Vi=12517, En=29345


In [2]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_head):
        super(MultiHeadAttention, self).__init__()
        self.d_head = d_model // n_head
        self.n_head = n_head
        self.d_model = d_model

        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.fc_out = nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]
        Q = self.w_q(query).view(batch_size, -1, self.n_head, self.d_head).permute(0, 2, 1, 3)
        K = self.w_k(key).view(batch_size, -1, self.n_head, self.d_head).permute(0, 2, 1, 3)
        V = self.w_v(value).view(batch_size, -1, self.n_head, self.d_head).permute(0, 2, 1, 3)

        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / math.sqrt(self.d_head)
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e9)

        attention = torch.softmax(energy, dim=-1)
        x = torch.matmul(attention, V)
        x = x.permute(0, 2, 1, 3).contiguous().view(batch_size, -1, self.d_model)
        return self.fc_out(x)

class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionwiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_head, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, n_head)
        self.ffn = PositionwiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask):
        _src = self.self_attn(src, src, src, src_mask)
        src = self.norm1(src + self.dropout(_src))
        _src = self.ffn(src)
        src = self.norm2(src + self.dropout(_src))
        return src

class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_head, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, n_head)
        self.cross_attn = MultiHeadAttention(d_model, n_head)
        self.ffn = PositionwiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        _trg = self.self_attn(trg, trg, trg, trg_mask)
        trg = self.norm1(trg + self.dropout(_trg))
        _trg = self.cross_attn(trg, enc_src, enc_src, src_mask)
        trg = self.norm2(trg + self.dropout(_trg))
        _trg = self.ffn(trg)
        trg = self.norm3(trg + self.dropout(_trg))
        return trg

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model=256, n_head=8, n_layer=3, d_ff=512, dropout=0.1, max_len=150):
        super(Transformer, self).__init__()
        self.encoder = nn.ModuleList([EncoderLayer(d_model, n_head, d_ff, dropout) for _ in range(n_layer)])
        self.decoder = nn.ModuleList([DecoderLayer(d_model, n_head, d_ff, dropout) for _ in range(n_layer)])

        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.trg_embedding = nn.Embedding(trg_vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_len)
        self.fc_out = nn.Linear(d_model, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = 0
        self.trg_pad_idx = 0

    def make_src_mask(self, src):
        return (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

    def make_trg_mask(self, trg):
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        trg_len = trg.shape[1]
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=trg.device)).bool()
        return trg_pad_mask & trg_sub_mask

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        src = self.dropout(self.pos_encoding(self.src_embedding(src)))
        for layer in self.encoder:
            src = layer(src, src_mask)

        trg = self.dropout(self.pos_encoding(self.trg_embedding(trg)))
        for layer in self.decoder:
            trg = layer(trg, src, trg_mask, src_mask)

        return self.fc_out(trg)

In [3]:
class NoamScheduler:
    def __init__(self, optimizer, d_model, warmup_steps=4000, factor=1.0):
        self.optimizer = optimizer
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.factor = factor
        self.step_num = 0

    def step(self):
        self.step_num += 1
        lr = self.factor * (self.d_model ** -0.5) * min(self.step_num ** -0.5, self.step_num * (self.warmup_steps ** -1.5))
        for p in self.optimizer.param_groups:
            p['lr'] = lr

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, padding_idx, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.padding_idx = padding_idx

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
            true_dist[:, self.padding_idx] = 0
            mask = torch.nonzero(target.data == self.padding_idx)
            if mask.dim() > 0: true_dist.index_fill_(0, mask.squeeze(), 0.0)
        return torch.mean(torch.sum(-true_dist * pred, dim=-1))

D_MODEL = 256
model = Transformer(len(src_vocab), len(trg_vocab), d_model=D_MODEL, n_head=8, n_layer=3).to(DEVICE)

def init_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)
model.apply(init_weights)

optimizer = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
scheduler = NoamScheduler(optimizer, d_model=D_MODEL, warmup_steps=4000)
criterion = LabelSmoothingLoss(len(trg_vocab), padding_idx=0, smoothing=0.1)

EPOCHS = 10
print(f"Starting Training Phase 5 (Safe Config)... Params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    start_time = time.time()

    for src, trg in train_loader:
        src, trg = src.to(DEVICE), trg.to(DEVICE)

        trg_input = trg[:, :-1]
        trg_output = trg[:, 1:]

        optimizer.zero_grad()
        output = model(src, trg_input)

        output = output.contiguous().view(-1, output.shape[-1])
        trg_output = trg_output.contiguous().view(-1)

        loss = criterion(output, trg_output)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1:02} | Time: {(time.time()-start_time)/60:.1f}m | Loss: {epoch_loss/len(train_loader):.4f}")

    # Save checkpoint
    torch.save(model.state_dict(), os.path.join(CHECKPOINT_PATH, 'phase5_safe_best.pt'))

Starting Training Phase 5 (Safe Config)... Params: 22,212,001
Epoch 01 | Time: 5.6m | Loss: 2.2363
Epoch 02 | Time: 5.7m | Loss: 1.9118
Epoch 03 | Time: 5.7m | Loss: 1.7639
Epoch 04 | Time: 5.7m | Loss: 1.6441
Epoch 05 | Time: 5.6m | Loss: 1.5742
Epoch 06 | Time: 5.7m | Loss: 1.5200
Epoch 07 | Time: 5.7m | Loss: 1.4793
Epoch 08 | Time: 5.7m | Loss: 1.4502
Epoch 09 | Time: 5.7m | Loss: 1.4258
Epoch 10 | Time: 5.7m | Loss: 1.4110


In [4]:
def beam_search_decode(model, src_sentence, src_vocab, trg_vocab, device, beam_width=3, max_len=50):
    model.eval()

    tokens = [1] + src_vocab.numericalize(src_sentence) + [2]
    src_tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)
    src_mask = model.make_src_mask(src_tensor)

    with torch.no_grad():
        src_emb = model.dropout(model.pos_encoding(model.src_embedding(src_tensor)))
        enc_src = src_emb
        for layer in model.encoder:
            enc_src = layer(enc_src, src_mask)

    sequences = [([1], 0.0, False)]

    for _ in range(max_len):
        all_candidates = []

        for seq, score, finished in sequences:
            if finished:
                all_candidates.append((seq, score, True))
                continue

            trg_tensor = torch.LongTensor(seq).unsqueeze(0).to(device)
            trg_mask = model.make_trg_mask(trg_tensor)

            with torch.no_grad():
                trg_emb = model.dropout(model.pos_encoding(model.trg_embedding(trg_tensor)))
                out = trg_emb
                for layer in model.decoder:
                    out = layer(out, enc_src, trg_mask, src_mask)
                output = model.fc_out(out)

            prob = output[:, -1, :].log_softmax(dim=-1)
            topk_probs, topk_idxs = prob.topk(beam_width)

            for i in range(beam_width):
                token = topk_idxs[0][i].item()
                token_prob = topk_probs[0][i].item()

                new_seq = seq + [token]
                new_score = score + token_prob

                if token == 2: # EOS
                    all_candidates.append((new_seq, new_score, True))
                else:
                    all_candidates.append((new_seq, new_score, False))

        ordered = sorted(all_candidates, key=lambda x: x[1] / (len(x[0])**0.7), reverse=True)
        sequences = ordered[:beam_width]

        if all([s[2] for s in sequences]):
            break

    best_seq = sequences[0][0]

    words = []
    for idx in best_seq:
        if idx not in [0, 1, 2, 3]:
            words.append(trg_vocab.itos[idx])

    return " ".join(words)

print("\n--- Testing Beam Search ---")
test_sentence = "Tôi là sinh viên"
print(f"SRC: {test_sentence}")
print(f"PRED: {beam_search_decode(model, test_sentence, src_vocab, trg_vocab, DEVICE, beam_width=3)}")


--- Testing Beam Search ---
SRC: Tôi là sinh viên
PRED: i 'm students .


In [5]:
!pip install -q sacrebleu tqdm
import sacrebleu
from tqdm import tqdm

print("Downloading Test Data...")
os.system("wget -q https://github.com/stefan-it/nmt-en-vi/raw/master/data/test-2013-en-vi.tgz -O test.tgz")
with tarfile.open("test.tgz", "r:gz") as tar: tar.extractall()

def clean_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [html.unescape(line).strip() for line in f]

test_src = clean_file('tst2013.vi')
test_ref = clean_file('tst2013.en')

hypotheses = []
print(f"Evaluating Beam Search on {len(test_src)} sentences...")

for sent in tqdm(test_src):
    pred = beam_search_decode(model, sent, src_vocab, trg_vocab, DEVICE, beam_width=3)
    hypotheses.append(pred)

bleu = sacrebleu.corpus_bleu(hypotheses, [test_ref], tokenize='13a')
print(f"\nFINAL BLEU SCORE: {bleu.score:.2f}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.8 MB/s eta 0:00:00


/tmp/ipython-input-4109901455.py:7: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  with tarfile.open("test.tgz", "r:gz") as tar: tar.extractall()


Evaluating Beam Search on 1268 sentences...


100%|██████████| 1268/1268 [05:58<00:00,  3.54it/s]



FINAL BLEU SCORE: 13.96
